In [ ]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
from typing import Sequence, Tuple, Union
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from functools import partial
from dataclasses import dataclass
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from IPython.display import display

import logging
import sys

sys.path.append('..')

{
  "shell_port": 65093,
  "iopub_port": 65094,
  "stdin_port": 65095,
  "control_port": 65096,
  "hb_port": 65097,
  "ip": "127.0.0.1",
  "key": "d63ad15b-75cc7a1a492548e62ff186b1",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-3a3d9037-09de-4da4-8f75-50655a73247d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned OFF
